# calculate mean and std

In [2]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class SimpleImageLoader(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, index):
        img_path = self.image_files[index]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

def compute_mean_std(loader):
    channel_sum, channel_sq_sum, num_batches = 0, 0, 0
    
    for images in loader:
        images = images.view(images.size(0), images.size(1), -1)
        channel_sum += torch.mean(images, dim=[0, 2])
        channel_sq_sum += torch.mean(images ** 2, dim=[0, 2])
        num_batches += 1

    mean = channel_sum / num_batches
    std = (channel_sq_sum / num_batches - mean ** 2) ** 0.5
    return mean, std

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create the dataset
dataset = SimpleImageLoader('dataset/train', transform=transform)
loader = DataLoader(dataset, batch_size=10, num_workers=0, shuffle=False)

# Calculate mean and std deviation
mean, std = compute_mean_std(loader)
print(f"Calculated Mean: {mean}")
print(f"Calculated Std Deviation: {std}")



Calculated Mean: tensor([0.6984, 0.5219, 0.4197])
Calculated Std Deviation: tensor([0.1396, 0.1318, 0.1236])


In [3]:
# Create the dataset
dataset = SimpleImageLoader('dataset/val', transform=transform)
loader = DataLoader(dataset, batch_size=10, num_workers=0, shuffle=False)

# Calculate mean and std deviation
mean, std = compute_mean_std(loader)
print(f"Calculated Mean: {mean}")
print(f"Calculated Std Deviation: {std}")

Calculated Mean: tensor([0.6982, 0.5217, 0.4194])
Calculated Std Deviation: tensor([0.1395, 0.1320, 0.1239])


# use least used gpu

In [4]:
import subprocess
import os

def get_least_used_gpu():
    # Command to get GPU usage (memory and compute)
    smi_output = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used,utilization.gpu', '--format=csv,noheader,nounits']).decode()

    # Parse the output to get a list of (memory used, gpu utilization) tuples
    gpu_stats = [tuple(map(int, line.split(', '))) for line in smi_output.strip().split('\n')]
    
    # Calculate a simple score by adding memory usage and GPU utilization (you can customize this)
    usage_scores = [memory + utilization for memory, utilization in gpu_stats]

    # Get the index of the GPU with the lowest score
    least_used_gpu = usage_scores.index(min(usage_scores))
    return least_used_gpu

# Set the least used GPU as visible
least_used_gpu = get_least_used_gpu()
os.environ['CUDA_VISIBLE_DEVICES'] = str(least_used_gpu)
print("Using GPU:", least_used_gpu)


Using GPU: 2


# Nan handling


In [5]:
import pandas as pd

# Load the CSV file
file_path = 'dataset/train-metadata_age_fixed.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Get the summary of NaN values
nan_summary = df.isna().sum()

# Display the summary
print("Summary of NaN values in each column:")
print(nan_summary)


/tmp/ipykernel_68766/2137989671.py:5: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Summary of NaN values in each column:
isic_id                              0
target                               0
patient_id                           0
age_approx                           0
sex                              11517
anatom_site_general               5756
clin_size_long_diam_mm               0
image_type                           0
tbp_tile_type                        0
tbp_lv_A                             0
tbp_lv_Aext                          0
tbp_lv_B                             0
tbp_lv_Bext                          0
tbp_lv_C                             0
tbp_lv_Cext                          0
tbp_lv_H                             0
tbp_lv_Hext                          0
tbp_lv_L                             0
tbp_lv_Lext                          0
tbp_lv_areaMM2                       0
tbp_lv_area_perim_ratio              0
tbp_lv_color_std_mean                0
tbp_lv_deltaA                        0
tbp_lv_deltaB                        0
tbp_lv_deltaL             

In [4]:
# Replace NaN values in the 'age' column with the mean of the 'age' column
df['age_approx'] = df['age_approx'].fillna(df['age_approx'].mean())

# Save the modified DataFrame to a new CSV file
output_file_path = 'dataset/train-metadata_age_fixed.csv'  # Replace with your desired output file path
df.to_csv(output_file_path, index=False)